In [1]:
import os
import json
import re
import shutil
input_folder = "encoded_yosys"
output_folder = "done_all"
# import torch
# import torch.nn.functional as F


In [2]:
# Use list comprehension to create a list of file paths
verilog_files = [os.path.join(input_folder, filename) for filename in os.listdir(input_folder) if filename.endswith('.txt')]

# Print the list of file paths
print(len(verilog_files))
print(type(verilog_files[0]))
print(verilog_files[0])


396
<class 'str'>
encoded_yosys\adder10_synth.txt


In [3]:
type_index = {
    "input":    "1000000",
    "output":   "0100000",
    "wire":     "0010000",
    "not" :     "0001000",
    "or" :      "0000100",
    "and":      "0000010",
    "constVal": "0000001"
}

In [4]:
files_dict = dict()
for file in verilog_files:
    num_nodes = 0
    num_edges = 0
    input_count = 0
    output_count = 0
    wire_count = 0
    or_count = 0
    and_count = 0
    not_count = 0
    constVal_count = 0
    file_name = file.split(".")[0].split("\\")[1]
    if file.endswith('.txt'):
        with open(file, "r") as f:
            loaded_data = json.load(f)
            num_nodes = len(loaded_data[0])
            num_edges = len(loaded_data[1][0])
            for sublist in loaded_data[0]:
                if sublist[0]:
                    input_count += 1
                if sublist[1]:
                    output_count += 1
                if sublist[2]:
                    wire_count += 1
                if sublist[3]:
                    not_count += 1
                if sublist[4]:
                    or_count += 1
                if sublist[5]:
                    and_count += 1
                if sublist[6]:
                    constVal_count += 1
                
            statistics_dic = {"num_nodes": num_nodes, "num_edges" : num_edges}
            count_dict = {"input_count": input_count, "output_count": output_count, "wire_count": wire_count, "not_count": not_count, "or_count": or_count, "and_count": and_count, "constVal_count": constVal_count}
            files_dict[file_name] = {"statistics":statistics_dic,"counts": count_dict}


In [5]:
with open("files_info.txt", 'w') as file:
    file.write(json.dumps(files_dict, indent=4))

In [6]:
with open("labels_dict.txt", 'r') as file:
    data = file.read()
data_dict = json.loads(data)
for key, value in data_dict.items():
    print(key, value)
    

adder 28
and 30
comparator 22
decoder 32
encoder 25
mult 29
mux 27
nand 31
nor 30
not 16
or 27
pe 21
sub 11
xnor 30
xor 28


In [7]:
num_nodes_files_list = []
num_edges_files_list = []
for file in verilog_files:
    with open(file, "r") as file:
        loaded_data = json.load(file)
        num_nodes_files_list.append(len(loaded_data[0]))
        num_edges_files_list.append(len(loaded_data[1][0]))

In [10]:
avergae_num_nodes = sum(num_nodes_files_list) / len(num_nodes_files_list)
max_num_nodes = max(num_nodes_files_list)
min_num_nodes = min(num_nodes_files_list)

print("Average number of nodes: ", avergae_num_nodes)
print("Max number of nodes: ", max_num_nodes)
print("Min number of nodes: ", min_num_nodes)

Average number of nodes:  55.30555555555556
Max number of nodes:  1736
Min number of nodes:  3


In [11]:
average_edges = sum(num_edges_files_list) / len(num_edges_files_list)
max_edges = max(num_edges_files_list)
min_edges = min(num_edges_files_list)

print("Average number of edges: ", average_edges)
print("Max number of edges: ", max_edges)
print("Min number of edges: ", min_edges)

Average number of edges:  77.61363636363636
Max number of edges:  2560
Min number of edges:  2


In [12]:
sum_nodes = sum(num_nodes_files_list)
sum_nodes

21901

In [13]:
files_dict = dict()
for key in data_dict.keys():
    num_nodes = 0
    num_edges = 0
    num_files = 0
    # node_sizes = []
    for file_path in verilog_files:
        st = file_path.split("\\")[1].split(".")[0]
        # print(st)
        pattern = f"^({key})\d+"
        # print(pattern)
        if re.match(pattern, st):
            # print("hi")
            num_files += 1
            with open(file_path, "r") as file:
                loaded_data = json.load(file)
                num_nodes += len(loaded_data[0])
                num_edges += len(loaded_data[1][0])
                # for sublist in loaded_data[0]:
                #     node_sizes.append(sublist[3])
                
    files_dict[key] = {
        "num_nodes": num_nodes,
        "num_edges": num_edges,
        "average_nodes": round(num_nodes / num_files,2),
        "average_edges": round(num_edges / num_files,2),
        "num_files": num_files,
        "percentage_nodes" : round(num_nodes / sum_nodes * 100,2),
        "percentage_edges" : round(num_edges / sum(num_edges_files_list) * 1000,2),
        "percentage_files" : round(num_files / len(verilog_files) * 100,2)
    }
# files_dict

In [15]:
encoded_files = [os.path.join(output_folder, filename) for filename in os.listdir(input_folder) if filename.endswith('.txt')]
encoded_files[0]

'done_all\\adder10_synth.txt'

In [16]:
nodes_files_info = dict()
input_count = 0
output_count = 0
reg_count = 0
wire_count = 0
constVal_count = 0
not_count = 0
or_count = 0
and_count = 0
for key in data_dict.keys():
    key_dict = dict()
    for file_path in encoded_files:
        st = file_path.split("\\")[1].split(".")[0].split("_")[0]
        # print(st)
        # print(key)
        pattern = f"^({key})\d+"
        # print(re.match(pattern, st))
        if re.match(pattern, st):
            with open(file_path, "r") as file:
                loaded_data = json.load(file)
                for sublist in loaded_data[0]:
                    # print(sublist)
                    # break
                    if sublist[0]: # input
                        key_dict["input"] = key_dict.get("input", 0) + 1
                        input_count +=1
                        
                    elif sublist[1]: #output
                        key_dict["output"] = key_dict.get("output", 0) + 1
                        output_count += 1
                        
                        
                    elif sublist[2]: #wire
                        key_dict["wire"] = key_dict.get("wire", 0) + 1
                        wire_count +=1
                        
                    elif sublist[3]:
                        key_dict["not"] = key_dict.get("not", 0) + 1
                        not_count+=1
                        
                    elif sublist[4]:
                        key_dict["or"] = key_dict.get("or", 0) + 1
                        or_count+=1
                        
                    elif sublist[5]:
                        key_dict["and"] = key_dict.get("and", 0) + 1
                        and_count+=1
                        
                    elif sublist[6]:
                        key_dict["constVal"] = key_dict.get("constVal", 0) + 1
                        constVal_count+=1
                        
                    else: 
                        print("eh el habal da")
                        
    nodes_files_info[key] = key_dict
                
    
# files_dict

In [17]:
print("Input Nodes Count = ", input_count)
print("Output Nodes Count = ", output_count)
print("Wire Nodes Count = ", wire_count)
print()
print("ConstVal Nodes Count = ", constVal_count)
print("Not Nodes Count = ", not_count)
print("Or Nodes Count = ", or_count)
print("And Nodes Count = ", and_count)

Input Nodes Count =  1211
Output Nodes Count =  538
Wire Nodes Count =  8938

ConstVal Nodes Count =  13
Not Nodes Count =  1782
Or Nodes Count =  3928
And Nodes Count =  4808


In [18]:
# write global statistics to file
with open("global_statistics.txt", "w") as file:
    file.write("Average number of nodes: " + str(avergae_num_nodes) + "\n")
    file.write("Max number of nodes: " + str(max_num_nodes) + "\n")
    file.write("Min number of nodes: " + str(min_num_nodes) + "\n")
    file.write("Average number of edges: " + str(average_edges) + "\n")
    file.write("Max number of edges: " + str(max_edges) + "\n")
    file.write("Min number of edges: " + str(min_edges) + "\n")
    file.write("Sum number of nodes: " + str(sum_nodes) + "\n")

In [19]:
with open("files_global_statistics.txt", "w") as file:
    file.write(json.dumps(files_dict, indent=4))

In [20]:
with open("global_dataset_nodes_info.txt", "w") as file:
    file.write("Input = " + str(input_count) + "\n")
    file.write("Output = " + str(output_count) + "\n")
    file.write("Wire = " + str(wire_count) + "\n")
    file.write("\n")
    file.write("ConstVal = " + str(constVal_count) + "\n")
    file.write("Not = " + str(not_count) + "\n")
    file.write("Or = " + str(or_count) + "\n")
    file.write("And = " + str(and_count) + "\n")

In [21]:
with open("classes_nodes_info.txt", "w") as file:
    file.write(json.dumps(nodes_files_info, indent=4))